In [1]:
!pip install transformers
!pip install langdetect
!pip install translatepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 19.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=0e1ba4e72c05c8ff8f4360731eabe8db492d57d713505217cea7d5a3fac67c9f
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 814.4/814.4 kB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 8.0 MB/s eta 0:00:00
  Created wheel for safeIO: filename=safeIO-1.2-py3-none-any.whl size=6732 sha256=aa4b7d340aa87cd1c3f7736949cdc84cfd4dbc0a8a891e651c98068d00a770e4
  Stored in directory: /root/.cache/pip/wheels/65/77/55/49296e806480db1fa92548fe2eb7447048830f14dbf3e34940
Successfully built safeIO


In [2]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import pickle
from langdetect import detect
import re
from translatepy import Translator
import numpy as np
import time

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
file_path = '/content/drive/My Drive/IVDA/zora_sustainable.pk'
# Pkl-Datei zu öffnen und zu laden:
with open(file_path, 'rb') as file:
    X = pickle.load(file)

In [39]:
X.describe()

,id_numerical,sdg1,sdg2,sdg3,sdg4,sdg5,sdg6,sdg7,sdg8,sdg9,...,17.10,17.11,17.12,17.13,17.14,17.15,17.16,17.17,17.18,17.19
count,27162.000000,27154.000000,27154.000000,27154.000000,27154.000000,27154.000000,27154.000000,27154.000000,27154.000000,27154.000000,...,27154.000000,27154.000000,27154.000000,27154.000000,27154.000000,27154.000000,27154.000000,27154.000000,27154.000000,27154.000000
mean,31928.563066,0.068010,0.207663,0.556007,0.154889,0.142430,0.226736,0.206214,0.174604,0.228393,...,0.035411,0.017705,0.483830,0.077355,0.024521,0.041968,0.076853,0.070366,0.066159,0.023411
std,20038.136147,0.196200,0.280599,0.433044,0.271695,0.268473,0.324477,0.318664,0.295725,0.284934,...,0.154417,0.063346,0.000121,0.201695,0.104589,0.136960,0.194784,0.197712,0.173312,0.084050
min,1.000000,0.003305,0.005008,0.002048,0.001378,0.001778,0.001677,0.001646,0.004653,0.008562,...,0.001594,0.007341,0.483164,0.004995,0.003852,0.012147,0.011760,0.006318,0.006421,0.008073
25%,13950.500000,0.003842,0.022421,0.032342,0.001640,0.002116,0.010986,0.002454,0.007196,0.010724,...,0.002615,0.008278,0.483800,0.006169,0.004363,0.012911,0.013017,0.007271,0.008507,0.010312
50%,31508.000000,0.003923,0.070286,0.734853,0.002062,0.003504,0.044574,0.017755,0.016334,0.088357,...,0.003255,0.008634,0.483853,0.006966,0.004466,0.013600,0.013558,0.007670,0.010794,0.011549
75%,50359.750000,0.004058,0.258271,0.994773,0.192858,0.115974,0.327084,0.292253,0.176678,0.374161,...,0.005190,0.009872,0.483905,0.013083,0.004601,0.015214,0.015264,0.008493,0.020208,0.013800
max,65413.000000,0.995284,0.996041,0.998797,0.998946,0.996800,0.997693,0.998541,0.992124,0.984601,...,0.997481,0.994375,0.484082,0.994981,0.995381,0.987470,0.983357,0.995516,0.996798,0.995453


In [41]:
X['description'].head(3)

12235    BACKGROUND: Studies published in non-English l...
13951    BACKGROUND: In the quest for public and privat...
20023    BACKGROUND: Enterobacter sakazakii is a foodbo...
Name: description, dtype: object

In [42]:
tokenizer_en = AutoTokenizer.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
model_en = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")

tokenizer_de = AutoTokenizer.from_pretrained("dbmdz/bert-base-german-cased")
model_de = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-base-german-cased")

tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/456 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/240k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dbmdz/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [43]:
nlp_en = pipeline("ner", model=model_en, tokenizer=tokenizer_en)
nlp_de = pipeline("ner", model=model_de, tokenizer=tokenizer_de)

In [44]:
def extract_entities_loc(nlp_result):
    entities = []
    current_entity = ""

    for entity in nlp_result:
        word = entity['word']
        #print(word)
        if entity['entity'].startswith('B-LOC'):
            if current_entity:  # Finish the previous entity
                entities.append(current_entity)
            current_entity = word  # Start a new entity
        elif entity['entity'].startswith('I-LOC'):
            if word.startswith('##'):
                current_entity += word[2:]  # Append without '##' and without space
            else:
                current_entity += " " + word  # Append with space

    if current_entity:  # Add the last entity to the list
        entities.append(current_entity)

    return entities

In [45]:
# Testing the pipeline
example = X['description'][2]
print(example)
nlp_result = nlp_en(example)
print(nlp_result)
extract_entities_loc(nlp_result)[0].lstrip()

Glacier Hazards, Permafrost Hazards, and Glacier Lake Outburst Floods in Mountain Areas: Processes, Assessment, Prevention, Mitigation; Vienna, Austria, 10–13 November 2009; Recent atmospheric warming is profoundly affecting high-mountain environments around the world. Glaciers are thinning and retreating, new and often unstable lakes are forming at glacier margins, other lakes are suddenly draining, and permafrost is degrading. These changes pose serious hazards to people and property in mountain valleys. Several tens of thousands of people were killed by landslides, floods, and debris flows from high-mountain regions during the twentieth century, and there is concern that such events will increase as temperatures warm through the 21st century.
[{'entity': 'I-MISC', 'score': 0.82228833, 'index': 16, 'word': 'Glacier', 'start': 41, 'end': 48}, {'entity': 'I-MISC', 'score': 0.69749224, 'index': 17, 'word': 'Lake', 'start': 49, 'end': 53}, {'entity': 'I-MISC', 'score': 0.5702734, 'index'

'Vienna Austria'

In [46]:
def extract_first_ten_words(text):
    words = re.findall(r'\b\w+\b', text)
    return ' '.join(words[:10])

def detect_language(text):
    try:
        return detect(text)
    except:
        return "Language recognition failed!"

In [47]:
# Example text
text = X['description'][2]

# Erste 10 Wörter extrahieren
first_ten_words = extract_first_ten_words(text)
print("First 10 words:", first_ten_words)

# Sprache erkennen
language = detect_language(first_ten_words)
print("Language of the text:", language)

First 10 words: Glacier Hazards Permafrost Hazards and Glacier Lake Outburst Floods in
Language of the text: en


In [48]:
def translate_to_english(text):
    translator = Translator()
    translation = translator.translate(text, "English")
    return translation.result

# Example text
text = X['description'][0]

# Translation
translated_text = translate_to_english(text)
print("Übersetzter Text:", translated_text)

Übersetzter Text: This article examines the transport calculation of top management consultations.We show that the selection of candidates is influenced by the specific production conditions under which consultations act.If the work of partners withdraws directly from the client and are the advisory services offered to trust, it is to be expected that the best advisors will not be chosen as partners.


In [ ]:
# Create new columns for language and location entities filling both columns with NaN values:
X['language'] = np.nan
X['location entities'] = np.nan

# Process the dataset:
start_time = time.time()
for i in range(0, X.shape[0]):
    abstract_text = X.at[i, 'description']
    # Extract the first words of the abstract text to determine the language of the abstract:
    first_words = extract_first_ten_words(abstract_text)
    language = detect_language(first_words)
    # If the language of the text is not English, translate it to English:
    if language != "en":
        abstract_text = translate_to_english(abstract_text)
    # Extract the location enities form the translated text using the english BERT NER:
    nlp_result = nlp_en(abstract_text)
    # Store the data in the dataframe:
    X.at[i, 'language'] = language  # Verwenden Sie .at für einen sicheren Zugriff
    location_entities = extract_entities_loc(nlp_result)
    if location_entities:
        X.at[i, 'location entities'] = location_entities[0].lstrip()

    # Update and print status information
    print("("+str(round(100*(i+1)/X.shape[0],1)) +"%) " +str(i+1) +"/" +str(X.shape[0]) +" samples processed.")

print("Success!")
end_time = time.time()
processing_time = end_time - start_time
print(processing_time)

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
(61.8%) 16774/27162 samples processed.
(61.8%) 16775/27162 samples processed.
(61.8%) 16776/27162 samples processed.
(61.8%) 16777/27162 samples processed.
(61.8%) 16778/27162 samples processed.
(61.8%) 16779/27162 samples processed.
(61.8%) 16780/27162 samples processed.
(61.8%) 16781/27162 samples processed.
(61.8%) 16782/27162 samples processed.
(61.8%) 16783/27162 samples processed.
(61.8%) 16784/27162 samples processed.
(61.8%) 16785/27162 samples processed.
(61.8%) 16786/27162 samples processed.
(61.8%) 16787/27162 samples processed.
(61.8%) 16788/27162 samples processed.
(61.8%) 16789/27162 samples processed.
(61.8%) 16790/27162 samples processed.
(61.8%) 16791/27162 samples processed.
(61.8%) 16792/27162 samples processed.
(61.8%) 16793/27162 samples processed.
(61.8%) 16794/27162 samples processed.
(61.8%) 16795/27162 samples processed.
(61.8%) 16796/27162 samples processed.
(61.8%) 16797/27162 samples processe

NoResult: ignored

In [4]:
new_file_path = '/content/drive/My Drive/IVDA/zora_sustainable_processed.pk'

# Save processed dataframe to new pickle file:
with open(new_file_path, 'wb') as file:
    pickle.dump(X, file)

print(f"Datei erfolgreich gespeichert unter: {new_file_path}")

NameError: ignored

In [10]:
# Open and load new Pkl-file:
new_file_path = '/content/drive/My Drive/IVDA/zora_sustainable_processed_new.pk'
with open(new_file_path, 'rb') as file:
    X_new = pickle.load(file)

In [17]:
X_new[X_new['language'].isna()].head()

,id,title,date,author,publisher,relation,description,collections,faculty,institute,...,17.12,17.13,17.14,17.15,17.16,17.17,17.18,17.19,language,location entities
22463,oai:www.zora.uzh.ch:4,"Serotypes, intimin variants and other virulenc...",2005,"Blanco, Miguel, Schumacher, Sandra, Tasara, Ta...",BioMed Central,https://www.zora.uzh.ch/id/eprint/4/,BACKGROUND: Enteropathogenic Escherichia coli ...,Vetsuisse Faculty: Institute of Food Safety an...,Vetsuisse Faculty,Institute of Food Safety and Hygiene,...,0.483829,0.006285,0.004324,0.012715,0.012985,0.007359,0.017871,0.014333,NaN,NaN
25083,oai:www.zora.uzh.ch:5,16S rRNA gene based analysis of Enterobacter s...,2004,"Lehner, Angelika, Tasara, Taurai, Stephan, Roger",BioMed Central,https://www.zora.uzh.ch/id/eprint/5/,BACKGROUND: E. sakazakii is considered to be a...,Vetsuisse Faculty: Institute of Food Safety an...,Vetsuisse Faculty,Institute of Food Safety and Hygiene,...,0.483838,0.006738,0.004446,0.014092,0.013012,0.007643,0.522610,0.013285,NaN,NaN
26932,oai:www.zora.uzh.ch:9,The taxonomy of Enterobacter sakazakii: propos...,2007,"Iversen, C, Lehner, Angelika, Mullane, N, Bidl...",BioMed Central,https://www.zora.uzh.ch/id/eprint/9/,BACKGROUND: Enterobacter sakazakii is an oppor...,Vetsuisse Faculty: Institute of Food Safety an...,Vetsuisse Faculty,Institute of Food Safety and Hygiene,...,0.483663,0.007981,0.004240,0.014723,0.014471,0.006527,0.042141,0.016754,NaN,NaN
23521,oai:www.zora.uzh.ch:14,Mouse hypoxia-inducible factor-1alpha is encod...,1998-05-01,"Wenger, R H, Rolfs, A, Spielmann, P, Zimmerman...",American Society of Hematology,https://www.zora.uzh.ch/id/eprint/14/,Hypoxic induction of erythropoietin (Epo) and ...,Faculty of Medicine: University Hospital Zuric...,Faculty of Medicine,University Hospital Zurich,...,0.483811,0.005599,0.004457,0.012762,0.013550,0.008698,0.009496,0.009588,NaN,NaN
24048,oai:www.zora.uzh.ch:15,Approach to endoscopic extraperitoneal radical...,2007,"Blana, Andreas, Straub, Markus, Wild, Peter J,...",BioMed Central,https://www.zora.uzh.ch/id/eprint/15/,BACKGROUND: We report our approach regarding t...,Faculty of Medicine: University Hospital Zuric...,Faculty of Medicine,University Hospital Zurich,...,0.483892,0.007015,0.004540,0.013842,0.012508,0.007245,0.340960,0.010747,NaN,NaN


In [23]:
X1 = X_new[X_new['language'].notna()]
X2 = X_new[X_new['language'].isna()]

In [28]:
sum(X2['description'].isna())

0

In [ ]:
# Process the dataset:
start_time = time.time()
for i in range(0, X2.shape[0]):
    abstract_text = X2.iloc[i]['description']
    # Extract the first words of the abstract text to determine the language of the abstract:
    first_words = extract_first_ten_words(abstract_text)
    language = detect_language(first_words)
    # If the language of the text is not English, translate it to English:
    if language != "en":
        abstract_text = translate_to_english(abstract_text)
    # Extract the location enities form the translated text using the english BERT NER:
    nlp_result = nlp_en(abstract_text)
    # Store the data in the dataframe:
    X2.at[i, 'language'] = language  # Verwenden Sie .at für einen sicheren Zugriff
    location_entities = extract_entities_loc(nlp_result)
    if location_entities:
        X2.at[i, 'location entities'] = location_entities[0].lstrip()

    # Update and print status information
    print("("+str(round(100*(i+1)/X2.shape[0],1)) +"%) " +str(i+1) +"/" +str(X2.shape[0]) +" samples processed.")

print("Success!")
end_time = time.time()
processing_time = end_time - start_time
print(processing_time)

<ipython-input-49-eb9ceef882b2>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X2.at[i, 'language'] = language  # Verwenden Sie .at für einen sicheren Zugriff


(0.0%) 1/5390 samples processed.
(0.0%) 2/5391 samples processed.
(0.1%) 3/5392 samples processed.
(0.1%) 4/5393 samples processed.
(0.1%) 5/5394 samples processed.
(0.1%) 6/5395 samples processed.
(0.1%) 7/5396 samples processed.
(0.1%) 8/5397 samples processed.
(0.2%) 9/5398 samples processed.
(0.2%) 10/5399 samples processed.
(0.2%) 11/5400 samples processed.
(0.2%) 12/5401 samples processed.
(0.2%) 13/5402 samples processed.
(0.3%) 14/5403 samples processed.
(0.3%) 15/5404 samples processed.
(0.3%) 16/5405 samples processed.
(0.3%) 17/5406 samples processed.
(0.3%) 18/5407 samples processed.
(0.4%) 19/5408 samples processed.
(0.4%) 20/5409 samples processed.
(0.4%) 21/5410 samples processed.
(0.4%) 22/5411 samples processed.
(0.4%) 23/5412 samples processed.
(0.4%) 24/5413 samples processed.
(0.5%) 25/5414 samples processed.
(0.5%) 26/5415 samples processed.
(0.5%) 27/5416 samples processed.
(0.5%) 28/5417 samples processed.
(0.5%) 29/5418 samples processed.
(0.6%) 30/5419 samples 

In [ ]:
new_file_path = '/content/drive/My Drive/IVDA/zora_sustainable_processed_part2.pk'

# Save processed dataframe to new pickle file:
with open(new_file_path, 'wb') as file:
    pickle.dump(X, file)